In [34]:
#!/usr/bin/env python3
import wmfdata as wmf
import pandas as pd
import matplotlib

matplotlib.style.use('ggplot')

spark = wmf.spark.get_custom_session(
    master="yarn",
    spark_config={
        "spark.driver.memory": "16g",
        "spark.dynamicAllocation.maxExecutors": 128,
        "spark.executor.memory": "16g",
        "spark.executor.cores": 4,
        "spark.sql.shuffle.partitions": 512
    }
)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
22/08/08 19:18:47 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/08/08 19:18:47 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
22/08/08 19:18:47 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
22/08/08 19:18:47 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
22/08/08 19:18:47 WARN Utils: Service 'sparkDriver' could not bind on port 12003. Attempting port 12004.
22/08/08 19:18:47 WARN Utils: Service 'sparkDriver' could not bind on port 12004. Attempting port 12005.
22/08/08 19:18:47 WARN Utils: Service 'sparkDriver' could not bind on port 12005. Attempting port 12006.
22/08/08 19:18:47 WARN Utils: Service 'sparkDriver' could not bind on port 12006. Attempting port 12007.
22/08/

In [18]:
query = """
WITH pageviews AS (
SELECT month, year, COUNT(*) AS num_pageviews
FROM wmf.pageview_hourly
WHERE agent_type = 'user'
AND project = 'en.wikipedia'
GROUP BY month, year
), 

new_accounts AS (
SELECT
    cast(substr(user_first_edit_timestamp,1,4) as int) as year, 
    cast(substr(user_first_edit_timestamp,6,2) as int) as month,
    COUNT(*) AS num_new_accounts
FROM wmf.mediawiki_user_history 
WHERE snapshot = '2022-07'
AND wiki_db = 'enwiki'
AND array_contains(user_groups,'bot') = FALSE
AND user_first_edit_timestamp IS NOT NULL
GROUP BY
    cast(substr(user_first_edit_timestamp,1,4) as int),
    cast(substr(user_first_edit_timestamp,6,2) as int)
),
    
articles_created AS (
SELECT
    cast(substr(start_timestamp ,1,4) as int) as year, 
    cast(substr(start_timestamp,6,2) as int) as month,
    COUNT(*) AS num_articles_created
FROM wmf.mediawiki_page_history 
WHERE snapshot = '2022-07'
AND page_namespace = 1
GROUP BY
    cast(substr(start_timestamp,1,4) as int),
    cast(substr(start_timestamp,6,2) as int)
),

articles_deleted AS (
SELECT
    cast(substr(end_timestamp ,1,4) as int) as year, 
    cast(substr(end_timestamp,6,2) as int) as month,
    COUNT(*) AS num_articles_deleted
FROM wmf.mediawiki_page_history 
WHERE snapshot = '2022-07'
AND page_namespace = 1
GROUP BY
    cast(substr(end_timestamp,1,4) as int),
    cast(substr(end_timestamp,6,2) as int)
),

num_articles AS (
SELECT
    articles_created.month,
    articles_created.year,
    COALESCE(articles_deleted.num_articles_deleted,0) AS articles_deleted,
    articles_created.num_articles_created,
    articles_created.num_articles_created-COALESCE(articles_deleted.num_articles_deleted,0) AS article_diff,
    SUM(articles_created.num_articles_created-COALESCE(articles_deleted.num_articles_deleted,0)) OVER (ORDER BY articles_created.year, articles_created.month) AS num_articles 
FROM articles_created
LEFT JOIN articles_deleted
ON (articles_created.month = articles_deleted.month AND articles_created.year = articles_deleted.year)
WHERE articles_created.month IS NOT NULL
AND articles_created.year IS NOT NULL
)

SELECT pageviews.month, pageviews.year, pageviews.num_pageviews, new_accounts.num_new_accounts, num_articles.num_articles
FROM pageviews
INNER JOIN new_accounts
ON pageviews.year = new_accounts.year AND pageviews.month = new_accounts.month
INNER JOIN num_articles
ON pageviews.year = num_articles.year AND pageviews.month = num_articles.month

"""

df = wmf.spark.run(query)
df

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
22/08/07 20:27:05 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.
22/08/07 20:27:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/07 20:27:18 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


,month,year,num_pageviews,num_new_accounts,num_articles
0,7,2018,6356664047,56756,25803728
1,8,2018,6289665286,59618,26099767
2,7,2020,6478241839,56579,30524897
3,12,2020,6616498273,51393,31246667
4,2,2018,6030892434,62961,24856599
...,...,...,...,...,...
83,8,2019,6206375530,55260,28226775
84,3,2017,6615775728,78009,22897713
85,9,2016,5937693760,63243,22217550
86,1,2022,6365364819,52232,33279155


In [19]:
result_df = df.copy()
min_year = result_df['year'].min()
result_df['wiki_age'] = result_df['year'].subtract(min_year).multiply(12).add(result_df['month'])
result_df = result_df.sort_values(['year','month'])

In [21]:
FILEPATH = '/home/jmads/datasets/momentum/pageview_new_accounts_8-7-22.csv'

result_df.to_csv(FILEPATH,index=False)